# Predicting Drug-Protein Binding using Machine Learning 

#### Goal: To predict whether small molecules are active or inactive against [this] protien using machine learning model.  

In [179]:
!pip install chembl_webresource_client

In [181]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [183]:
target = new_client.target
her2 = target.search('HER2')
df = pd.DataFrame.from_dict(her2)
df


,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Homo sapiens,FASN/HER2,17.0,False,CHEMBL4106134,"[{'accession': 'P04626', 'component_descriptio...",PROTEIN COMPLEX,9606
1,[],Homo sapiens,Receptor tyrosine-protein kinase erbB-2,15.0,False,CHEMBL1824,"[{'accession': 'P04626', 'component_descriptio...",SINGLE PROTEIN,9606
2,[],Homo sapiens,von Hippel-Lindau disease tumor suppressor/Rec...,13.0,False,CHEMBL5465227,"[{'accession': 'P04626', 'component_descriptio...",PROTEIN-PROTEIN INTERACTION,9606
3,[],Homo sapiens,Epidermal growth factor receptor and ErbB2 (HE...,12.0,False,CHEMBL2111431,"[{'accession': 'P04626', 'component_descriptio...",PROTEIN FAMILY,9606
4,[],Homo sapiens,ErbB-2/ErbB-3 heterodimer,12.0,False,CHEMBL4630723,"[{'accession': 'P04626', 'component_descriptio...",PROTEIN COMPLEX,9606
5,[],Homo sapiens,Epidermal growth factor receptor,9.0,False,CHEMBL2363049,"[{'accession': 'P04626', 'component_descriptio...",PROTEIN FAMILY,9606


In [185]:
HER2 = df.loc[4, 'target_chembl_id']
HER2

'CHEMBL4630723'

In [187]:
act = new_client.activity
result = act.filter(target_chembl_id = HER2).filter(standard_type = 'DC50')

df_1 = pd.DataFrame.from_dict(result)
df_1.head()

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,"{'action_type': 'DEGRADER', 'description': 'Bi...",None,25620628,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5363577,Induction of HER2/HER3 degradation in human MD...,B,None,None,BAO_0000179,...,Homo sapiens,ErbB-2/ErbB-3 heterodimer,9606,None,None,DC50,uM,UO_0000064,None,0.52
1,"{'action_type': 'DEGRADER', 'description': 'Bi...",None,25620630,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5363578,Induction of HER2/HER3 degradation in human MD...,B,None,None,BAO_0000179,...,Homo sapiens,ErbB-2/ErbB-3 heterodimer,9606,None,None,DC50,uM,UO_0000064,None,0.584


In [233]:
df_2 = df_1[['molecule_chembl_id', 'standard_type', 'canonical_smiles', 'standard_units', 'standard_value', 'type',  'target_organism']]
df_2

,molecule_chembl_id,standard_type,canonical_smiles,standard_units,standard_value,type,target_organism
0,CHEMBL5440662,DC50,CC[n+]1c(/C=C/c2ccc(-c3ccc4ncnc(Nc5ccc(OCc6ccc...,uM,0.52,DC50,Homo sapiens
1,CHEMBL5440662,DC50,CC[n+]1c(/C=C/c2ccc(-c3ccc4ncnc(Nc5ccc(OCc6ccc...,uM,0.584,DC50,Homo sapiens


In [235]:
df_2.shape

(2, 7)

In [237]:
df_clean = df_2.dropna(subset = 'standard_value')
df_clean


,molecule_chembl_id,standard_type,canonical_smiles,standard_units,standard_value,type,target_organism
0,CHEMBL5440662,DC50,CC[n+]1c(/C=C/c2ccc(-c3ccc4ncnc(Nc5ccc(OCc6ccc...,uM,0.52,DC50,Homo sapiens
1,CHEMBL5440662,DC50,CC[n+]1c(/C=C/c2ccc(-c3ccc4ncnc(Nc5ccc(OCc6ccc...,uM,0.584,DC50,Homo sapiens


In [239]:
df_clean['standard_units'].value_counts


<bound method IndexOpsMixin.value_counts of 0    uM
1    uM
Name: standard_units, dtype: object>

In [247]:
df_clean['standard_units'] = 'nM'
df_clean['standard_value'] = df_clean['standard_value'] * 1000 
df_clean.head()

,molecule_chembl_id,standard_type,canonical_smiles,standard_units,standard_value,type,target_organism
0,CHEMBL5440662,DC50,CC[n+]1c(/C=C/c2ccc(-c3ccc4ncnc(Nc5ccc(OCc6ccc...,nM,0.520.520.520.520.520.520.520.520.520.520.520....,DC50,Homo sapiens
1,CHEMBL5440662,DC50,CC[n+]1c(/C=C/c2ccc(-c3ccc4ncnc(Nc5ccc(OCc6ccc...,nM,0.5840.5840.5840.5840.5840.5840.5840.5840.5840...,DC50,Homo sapiens


In [ ]:
Her2_target = []
for i in df_final.standard_value:
    if float(i) >= 10000:
        Her2_target.append('inactive')
    elif float(i) <= 1000:
        Her2_target.append('active')
    else:
        Her2_append('intermediate') 

In [ ]:
df_final.to_csv('HER2_target.csv', index = False)